In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style("whitegrid")

In [2]:
data1 = pd.read_csv('fraud_cases.csv')
data1.head()

,ClaimID,Date_Occurred
0,768000066,20120707
1,709296250,20120711
2,726714358,20120712
3,520593970,20120716
4,102695400,20120726


In [3]:
## This data will be used for training the model
data2 = pd.read_csv('ClaimHistory_training.csv')
data2.head()

,sys_sector,sys_process,sys_product,sys_dataspecification_version,sys_claimid,sys_currency_code,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,ph_name,policy_fleet_flag,policy_insured_amount,policy_profitability
0,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-338957796-02,EUR,2433.0,Collision,20121022,20121127,1,VOLKSWAGEN,2008.0,Teresita,F,Sanchez,0,74949.0,Low
1,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-434911509-02,EUR,3791.0,Collision,20140612,20140618,1,CITROEN,2003.0,Charanaka,M,Yim,0,100648.0,Very low
2,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-615568027-02,EUR,452.0,Collision,20130506,20130923,1,RENAULT,2001.0,Wisaam,L,el-Kalil,0,NaN,Low
3,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-917387010-02,EUR,555.0,Collision,20171112,20171206,1,RENAULT,2017.0,Dillon,M,Hilfiker,0,121081.0,High
4,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-281513737-02,EUR,382.0,Collision,20151021,20151202,1,BMW,2011.0,Carmen,F,Lovelace,0,94538.0,Very high


In [4]:
data2.sys_claimid.unique()

array(['MTR-338957796-02', 'MTR-434911509-02', 'MTR-615568027-02', ...,
       'MTR-591101349-02', 'MTR-790242664-02', 'MTR-815745885-02'],
      dtype=object)

In [5]:
# converting the claim id into a string to extract the desired numbers from the id
# checking the length of claim id to make sure all the claim id's have the same length
data2['len_claimid']=  data2['sys_claimid'].str.len() 
data2['len_claimid'].unique()

array([16])

In [6]:
# extracting the sub string from claim id to concatinate with fraud data
data2['new_claimid'] = data2['sys_claimid'].str[4:13]
data2['new_claimid'].head()

0    338957796
1    434911509
2    615568027
3    917387010
4    281513737
Name: new_claimid, dtype: object

In [7]:
# Merging the data using left join on new_claimid and ClaimID
train_data = data2.merge(data1,how="left",left_on="new_claimid",right_on="ClaimID")

In [8]:
train_data.head()

,sys_sector,sys_process,sys_product,sys_dataspecification_version,sys_claimid,sys_currency_code,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,...,ph_firstname,ph_gender,ph_name,policy_fleet_flag,policy_insured_amount,policy_profitability,len_claimid,new_claimid,ClaimID,Date_Occurred
0,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-338957796-02,EUR,2433.0,Collision,20121022,20121127,...,Teresita,F,Sanchez,0,74949.0,Low,16,338957796,NaN,NaN
1,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-434911509-02,EUR,3791.0,Collision,20140612,20140618,...,Charanaka,M,Yim,0,100648.0,Very low,16,434911509,NaN,NaN
2,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-615568027-02,EUR,452.0,Collision,20130506,20130923,...,Wisaam,L,el-Kalil,0,NaN,Low,16,615568027,NaN,NaN
3,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-917387010-02,EUR,555.0,Collision,20171112,20171206,...,Dillon,M,Hilfiker,0,121081.0,High,16,917387010,NaN,NaN
4,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-281513737-02,EUR,382.0,Collision,20151021,20151202,...,Carmen,F,Lovelace,0,94538.0,Very high,16,281513737,NaN,NaN


In [9]:
# Checking Claim IDs of fraud data and train data to check where merging was unsuccefull
set(data1.ClaimID.unique())-set(train_data.ClaimID.unique())

{'355905469/622895187', '453637761/522337148', '930746044/840746503'}

In [10]:
# Checking the IDs in fraud data(data1)
data1[data1.ClaimID.isin(['355905469/622895187', '453637761/522337148', '930746044/840746503'])]

,ClaimID,Date_Occurred
20,355905469/622895187,20120929
51,930746044/840746503,20130102
597,453637761/522337148,20171025


In [11]:
# Checking the data for these claim ids in train data
# Putting the IDs in a variable to check the
Missing_ID = ['355905469','622895187', '453637761','522337148', '930746044','840746503']

In [12]:
# Checking the new clain ID for the Missing IDs
train_data[train_data.new_claimid.isin(Missing_ID)]

,sys_sector,sys_process,sys_product,sys_dataspecification_version,sys_claimid,sys_currency_code,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,...,ph_firstname,ph_gender,ph_name,policy_fleet_flag,policy_insured_amount,policy_profitability,len_claimid,new_claimid,ClaimID,Date_Occurred
3127,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-930746044-02,EUR,1285.0,Theft,20130102,20130104,...,Katelyn,F,Keyes,0,NaN,Very high,16,930746044,NaN,NaN
3447,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-840746503-02,EUR,118.0,Collision,20130104,20130130,...,Paul,L,Kunzman,0,102157.0,High,16,840746503,NaN,NaN
10438,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-522337148-02,EUR,159.0,Collision,20171031,20171230,...,Hugo,M,Lepure,0,76308.0,Neutral,16,522337148,NaN,NaN
11324,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-622895187-02,EUR,374.0,Weather,20121001,20121102,...,Christian,M,Rivera,0,NaN,Low,16,622895187,NaN,NaN
46960,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-355905469-02,EUR,4292.0,Collision,20120929,20121208,...,Jennifer,L,Gonzales,1,NaN,Low,16,355905469,NaN,NaN
59336,Private NonLife,Claims_initial_load,MOTOR,4.5,MTR-453637761-02,EUR,1208.0,Collision,20171025,20171118,...,Asia,F,Zerom,0,42500.0,High,16,453637761,NaN,NaN


In [13]:
# changing datatype of "claim_date_reported" column
train_data['claim_date_reported'] = pd.to_datetime(train_data.claim_date_reported.astype(str), format='%Y%m%d')

In [14]:
# Maximum possible date is '20240809' for this analysis as claim can't be raise in future dates
train_data['claim_date_reported_new'] =  pd.to_datetime('20240809')
train_data['claim_date_reported_new']= np.where( train_data.claim_date_reported > pd.to_datetime('20240809'), train_data['claim_date_reported_new'],train_data['claim_date_reported'])

In [15]:
# Calculating the difference between reported date and occurance date
train_data['diff_days_between_claim_occ_report']= (train_data.claim_date_reported_new  - pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d'))/np.timedelta64(1, 'D')

In [16]:
# Checking all the date columns for irregualrites
train_data[['claim_date_reported','claim_date_reported_new','claim_date_occurred','diff_days_between_claim_occ_report']].sort_values('diff_days_between_claim_occ_report',ascending=False).head(5)

,claim_date_reported,claim_date_reported_new,claim_date_occurred,diff_days_between_claim_occ_report
5838,2023-05-04,2023-05-04,20150925,2778.0
3045,2022-12-07,2022-12-07,20150901,2654.0
1140,2021-10-29,2021-10-29,20140829,2618.0
3764,2021-06-17,2021-06-17,20140419,2616.0
706,2024-08-09,2024-08-09,20170627,2600.0


In [17]:
# dropping the coulmns with only 1 unique value
drop_col= ['sys_sector' ,'sys_process' ,'sys_product','sys_dataspecification_version','sys_currency_code']
train_data.drop(columns=drop_col, axis = 1,inplace=True)

In [18]:
train_data[train_data.object_year_construction > pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d').dt.year][['claim_date_occurred','object_year_construction']]

,claim_date_occurred,object_year_construction
159,20160219,2017.0
415,20140830,2016.0
959,20130510,2014.0
1912,20130409,2017.0
2012,20140627,2015.0
...,...,...
79947,20170811,2018.0
79958,20121228,2014.0
79970,20130828,2014.0
79978,20140516,2016.0


In [19]:
# Calculating the new year of construction
train_data['object_year_construction_new']= np.where( train_data.object_year_construction >pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d').dt.year,pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d').dt.year,train_data['object_year_construction'])
train_data[train_data.object_year_construction >pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d').dt.year][['claim_date_occurred','object_year_construction',"object_year_construction_new"]]

,claim_date_occurred,object_year_construction,object_year_construction_new
159,20160219,2017.0,2016.0
415,20140830,2016.0,2014.0
959,20130510,2014.0,2013.0
1912,20130409,2017.0,2013.0
2012,20140627,2015.0,2014.0
...,...,...,...
79947,20170811,2018.0,2017.0
79958,20121228,2014.0,2012.0
79970,20130828,2014.0,2013.0
79978,20140516,2016.0,2014.0


In [20]:
## checking for the rows with claim_amount_claimed_total > policy_insured_amount
train_data[train_data.claim_amount_claimed_total > train_data.policy_insured_amount]

,sys_claimid,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,...,policy_fleet_flag,policy_insured_amount,policy_profitability,len_claimid,new_claimid,ClaimID,Date_Occurred,claim_date_reported_new,diff_days_between_claim_occ_report,object_year_construction_new
275,MTR-623984743-02,127303.0,Collision,20170212,2017-02-22,1,OTHER,2012.0,Othell,M,...,0,95739.0,Neutral,16,623984743,NaN,NaN,2017-02-22,10.0,2012.0
1266,MTR-315891252-02,104320.0,Collision,20121231,2013-05-12,0,AUDI,2007.0,Keeley,F,...,0,81287.0,Very low,16,315891252,NaN,NaN,2013-05-12,132.0,2007.0
1455,MTR-329478529-02,128001.0,Weather,20120831,2012-09-14,1,RENAULT,1995.0,Kristopher,M,...,0,81444.0,Low,16,329478529,329478529,20120831.0,2012-09-14,14.0,1995.0
2033,MTR-416489525-02,173415.0,Other,20131218,2013-12-24,1,OTHER,2014.0,Christine,L,...,0,71047.0,High,16,416489525,NaN,NaN,2013-12-24,6.0,2013.0
2114,MTR-444711600-02,209933.0,Theft,20140930,2014-10-26,1,RENAULT,2004.0,Anissa,F,...,0,124622.0,Neutral,16,444711600,NaN,NaN,2014-10-26,26.0,2004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79346,MTR-254647016-02,44509.0,Collision,20140803,2014-08-03,1,BMW,2014.0,Matthew,M,...,1,30762.0,Low,16,254647016,NaN,NaN,2014-08-03,0.0,2014.0
79378,MTR-351980250-02,52061.0,Collision,20140401,2014-04-10,0,VOLKSWAGEN,2012.0,Monica,L,...,0,46980.0,Neutral,16,351980250,NaN,NaN,2014-04-10,9.0,2012.0
79465,MTR-899544708-02,69690.0,Collision,20170217,2017-02-19,0,RENAULT,2006.0,Carly,F,...,0,38966.0,Low,16,899544708,NaN,NaN,2017-02-19,2.0,2006.0
79573,MTR-925632201-02,67161.0,Weather,20170201,2017-02-03,0,OTHER,2002.0,Rayhan,M,...,1,53111.0,Neutral,16,925632201,NaN,NaN,2017-02-03,2.0,2002.0


In [21]:
train_data['claim_amount_claimed_total_new']= np.where( train_data.claim_amount_claimed_total >train_data.policy_insured_amount,train_data.policy_insured_amount,train_data['claim_amount_claimed_total'])
#checking if the correction worked
train_data[train_data.claim_amount_claimed_total >train_data.policy_insured_amount][["claim_amount_claimed_total","policy_insured_amount","claim_amount_claimed_total_new"]]

,claim_amount_claimed_total,policy_insured_amount,claim_amount_claimed_total_new
275,127303.0,95739.0,95739.0
1266,104320.0,81287.0,81287.0
1455,128001.0,81444.0,81444.0
2033,173415.0,71047.0,71047.0
2114,209933.0,124622.0,124622.0
...,...,...,...
79346,44509.0,30762.0,30762.0
79378,52061.0,46980.0,46980.0
79465,69690.0,38966.0,38966.0
79573,67161.0,53111.0,53111.0


In [22]:
# checking the claim id
train_data["is_duplicate"]= train_data['sys_claimid'].duplicated()
train_data[train_data["is_duplicate"]==True]

,sys_claimid,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,...,policy_profitability,len_claimid,new_claimid,ClaimID,Date_Occurred,claim_date_reported_new,diff_days_between_claim_occ_report,object_year_construction_new,claim_amount_claimed_total_new,is_duplicate
55974,MTR-491813645-02,218.0,Weather,20121206,2012-12-16,0,CITROEN,2012.0,Yvette,F,...,High,16,491813645,NaN,NaN,2012-12-16,10.0,2012.0,218.0,True
67106,MTR-603112190-02,777.0,Collision,20160716,2016-07-18,0,CITROEN,2012.0,Jazmyn,F,...,Very low,16,603112190,NaN,NaN,2016-07-18,2.0,2012.0,777.0,True


In [23]:
#getting data for duplicate ids
train_data[train_data["new_claimid"].isin(['491813645','603112190'])]

,sys_claimid,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,...,policy_profitability,len_claimid,new_claimid,ClaimID,Date_Occurred,claim_date_reported_new,diff_days_between_claim_occ_report,object_year_construction_new,claim_amount_claimed_total_new,is_duplicate
7742,MTR-491813645-02,2252.0,Collision,20160711,2017-01-07,1,BMW,2012.0,Ariyona,F,...,Neutral,16,491813645,NaN,NaN,2017-01-07,180.0,2012.0,2252.0,False
15915,MTR-603112190-02,235.0,Collision,20150425,2015-05-17,1,CITROEN,2013.0,Jalen,L,...,Very low,16,603112190,NaN,NaN,2015-05-17,22.0,2013.0,235.0,False
55974,MTR-491813645-02,218.0,Weather,20121206,2012-12-16,0,CITROEN,2012.0,Yvette,F,...,High,16,491813645,NaN,NaN,2012-12-16,10.0,2012.0,218.0,True
67106,MTR-603112190-02,777.0,Collision,20160716,2016-07-18,0,CITROEN,2012.0,Jazmyn,F,...,Very low,16,603112190,NaN,NaN,2016-07-18,2.0,2012.0,777.0,True


In [24]:
# For now,we keep the claim id with which the claim occured first
train_data.sort_values("claim_date_occurred", inplace = True) 
print("Number of rows in train data before removing duplicate ids:", train_data.shape[0])
train_data.drop_duplicates(['new_claimid'], keep = 'first', inplace = True) 
print("Number of rows in train data after removing duplicate ids:", train_data.shape[0])

Number of rows in train data before removing duplicate ids: 80000
Number of rows in train data after removing duplicate ids: 79998


In [25]:
# Age of vehicle when claim the occured(in years)
train_data['Age_of_vehicle']= (pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d').dt.year)-(train_data.object_year_construction_new)
train_data.head()

,sys_claimid,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,...,len_claimid,new_claimid,ClaimID,Date_Occurred,claim_date_reported_new,diff_days_between_claim_occ_report,object_year_construction_new,claim_amount_claimed_total_new,is_duplicate,Age_of_vehicle
8380,MTR-341919822-02,2263.0,Weather,20120701,2012-07-23,1,OTHER,2004.0,Mahmood,M,...,16,341919822,NaN,NaN,2012-07-23,22.0,2004.0,2263.0,False,8.0
65865,MTR-876039500-02,70.0,Collision,20120701,2012-07-01,0,VOLKSWAGEN,2011.0,Haifaaa,L,...,16,876039500,NaN,NaN,2012-07-01,0.0,2011.0,70.0,False,1.0
2113,MTR-573586408-02,653.0,Other,20120701,2012-07-21,0,AUDI,2002.0,Natacha,F,...,16,573586408,NaN,NaN,2012-07-21,20.0,2002.0,653.0,False,10.0
27671,MTR-144279619-02,9349.0,Collision,20120701,2012-07-01,1,RENAULT,2012.0,Amy,L,...,16,144279619,NaN,NaN,2012-07-01,0.0,2012.0,9349.0,False,0.0
13793,MTR-225054348-02,600.0,Theft,20120701,2012-07-15,0,CITROEN,2005.0,Triana,F,...,16,225054348,NaN,NaN,2012-07-15,14.0,2005.0,600.0,False,7.0


In [27]:
 #Time since claim reported as of '20240809'(in years)
train_data['time_since_claim']= (pd.to_datetime('20240809')  - pd.to_datetime(train_data.claim_date_occurred.astype(str), format='%Y%m%d'))/np.timedelta64(1,'Y') 

In [28]:
# creating label column in the train dataset
train_data['label']= np.where(train_data.new_claimid.isin(train_data.ClaimID),1,0)
train_data['label'].value_counts() * 100 /len(train_data)
# current fraud rate is .76%

0    99.231231
1     0.768769
Name: label, dtype: float64

In [29]:
#creating a fraud flag
train_data["Fraud"] = np.where(train_data.label==1,"Fraud","Non_fraud")

In [30]:
# Converting Null and L into 'other' column
train_data['ph_gender'] = train_data['ph_gender'].apply(lambda x : 'other' if x == 'L' or x == 'missing' else x)

In [31]:
train_data = train_data.drop('len_claimid',axis=1)
train_data.head()

,sys_claimid,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,...,Date_Occurred,claim_date_reported_new,diff_days_between_claim_occ_report,object_year_construction_new,claim_amount_claimed_total_new,is_duplicate,Age_of_vehicle,time_since_claim,label,Fraud
8380,MTR-341919822-02,2263.0,Weather,20120701,2012-07-23,1,OTHER,2004.0,Mahmood,M,...,NaN,2012-07-23,22.0,2004.0,2263.0,False,8.0,12.107025,0,Non_fraud
65865,MTR-876039500-02,70.0,Collision,20120701,2012-07-01,0,VOLKSWAGEN,2011.0,Haifaaa,other,...,NaN,2012-07-01,0.0,2011.0,70.0,False,1.0,12.107025,0,Non_fraud
2113,MTR-573586408-02,653.0,Other,20120701,2012-07-21,0,AUDI,2002.0,Natacha,F,...,NaN,2012-07-21,20.0,2002.0,653.0,False,10.0,12.107025,0,Non_fraud
27671,MTR-144279619-02,9349.0,Collision,20120701,2012-07-01,1,RENAULT,2012.0,Amy,other,...,NaN,2012-07-01,0.0,2012.0,9349.0,False,0.0,12.107025,0,Non_fraud
13793,MTR-225054348-02,600.0,Theft,20120701,2012-07-15,0,CITROEN,2005.0,Triana,F,...,NaN,2012-07-15,14.0,2005.0,600.0,False,7.0,12.107025,0,Non_fraud


In [32]:
## Using the average of the column to fill the na values
train_data['policy_insured_amount'] = train_data['policy_insured_amount'].fillna(np.mean(train_data['policy_insured_amount']))

In [33]:
train_data.head()

,sys_claimid,claim_amount_claimed_total,claim_causetype,claim_date_occurred,claim_date_reported,claim_location_urban_area,object_make,object_year_construction,ph_firstname,ph_gender,...,Date_Occurred,claim_date_reported_new,diff_days_between_claim_occ_report,object_year_construction_new,claim_amount_claimed_total_new,is_duplicate,Age_of_vehicle,time_since_claim,label,Fraud
8380,MTR-341919822-02,2263.0,Weather,20120701,2012-07-23,1,OTHER,2004.0,Mahmood,M,...,NaN,2012-07-23,22.0,2004.0,2263.0,False,8.0,12.107025,0,Non_fraud
65865,MTR-876039500-02,70.0,Collision,20120701,2012-07-01,0,VOLKSWAGEN,2011.0,Haifaaa,other,...,NaN,2012-07-01,0.0,2011.0,70.0,False,1.0,12.107025,0,Non_fraud
2113,MTR-573586408-02,653.0,Other,20120701,2012-07-21,0,AUDI,2002.0,Natacha,F,...,NaN,2012-07-21,20.0,2002.0,653.0,False,10.0,12.107025,0,Non_fraud
27671,MTR-144279619-02,9349.0,Collision,20120701,2012-07-01,1,RENAULT,2012.0,Amy,other,...,NaN,2012-07-01,0.0,2012.0,9349.0,False,0.0,12.107025,0,Non_fraud
13793,MTR-225054348-02,600.0,Theft,20120701,2012-07-15,0,CITROEN,2005.0,Triana,F,...,NaN,2012-07-15,14.0,2005.0,600.0,False,7.0,12.107025,0,Non_fraud
